https://www.pyimagesearch.com/2016/01/11/opencv-panorama-stitching/

In [1]:
# import the necessary packages
import numpy as np
import imutils
import cv2
from functools import reduce
import os
from pathlib import Path
import pickle

In [17]:
def in_interval(o, w0, w1, h0, h1):
    '''
    o is a tuple like (79,153). w0, w1, h0, h1 defines a square.
    if o in the square, return True
    '''
    return (o[0] >= w0) and (o[0] <= w1) and (o[1] >= h0) and (o[1] <= h1)


def grid_pick(querykeys,trainkeys):
#         '''
#         input:feature points(querykeys, trainkeys)
#         output:selected points(querykeys, trainkeys)
#         原理；对参与拼接的第二张图（因为第一张图有可能是多图拼的，图像区域不规则）
#         划分grid_szw*grid_szh网格，在不同格子里提取(trainkeys)特征点，然后对应选出
#         第一张图的相应点，最后返回筛选后的querykeys, trainkeys
#         '''
    grid_szh = 5
    grid_szw = 5
    RESIZE_H=1000
    RESIZE_W=750
    inter_h = [(RESIZE_H // grid_szh) * i for i in range(grid_szh)]
    inter_h.append(RESIZE_H)
    inter_w = [(RESIZE_W // grid_szw) * i for i in range(grid_szw)]
    inter_w.append(RESIZE_W)
    res_train = list()
    res_query = list()
    tmplist = np.zeros([grid_szh, grid_szw, 2])  # 记录像素点的位置flag
    for i in range(grid_szw):
        for j in range(grid_szh):
            for o in trainkeys:  # 下面的判断保证每个方格只取一个点
                if ((in_interval(o, inter_w[i], inter_w[i + 1], inter_h[j], inter_h[j + 1])) and (
                        np.array(tmplist[j][i]).all() == np.array([0, 0]).all())):
                    res_train.append(o)
                    tmplist[j][i] = o
                    res_query.append(querykeys[trainkeys.index(o)])  # 如果trainkeys的某格子里有一点，把对应位置querykeys也加上
    return (res_query, res_train)

###########################
#下面都是从panorama.py里复制的函数

def make_mask(target, src):
    CommonMaskRGB = np.zeros((target.shape[0], target.shape[1]), dtype=np.uint8)
    SrcMaskRGB = np.zeros((target.shape[0], target.shape[1]), dtype=np.uint8)
    TargetMaskRGB = np.zeros((target.shape[0], target.shape[1]), dtype=np.uint8)
    CommonMaskRGB[(cv2.cvtColor(target,cv2.COLOR_RGB2GRAY) != 0) * (cv2.cvtColor(src,cv2.COLOR_RGB2GRAY) != 0)] = 255
    SrcMaskRGB[(cv2.cvtColor(src,cv2.COLOR_RGB2GRAY) != 0) * (CommonMaskRGB == 0)] = 255
    TargetMaskRGB[(cv2.cvtColor(target,cv2.COLOR_RGB2GRAY) != 0)] = 255
    CommonMask = cv2.erode(CommonMaskRGB,np.ones((5,5),np.uint8),iterations = 3)
    SrcMask = cv2.dilate(SrcMaskRGB,np.ones((5,5),np.uint8),iterations = 1)
    TargetMask = cv2.dilate(TargetMaskRGB,np.ones((3,3),np.uint8),iterations = 1)
    return CommonMask, SrcMask, TargetMask


def calc_dst4points(H, size):
    x = []
    y = []
    x.append(((H[0][0]*0 + H[0][1]*0 + H[0][2])/(H[2][0]*0 + H[2][1]*0 + H[2][2])))
    y.append(((H[1][0]*0 + H[1][1]*0 + H[1][2])/(H[2][0]*0 + H[2][1]*0 + H[2][2])))
    x.append(((H[0][0]*0 + H[0][1]*size[0] + H[0][2])/(H[2][0]*0 + H[2][1]*size[0] + H[2][2])))
    y.append(((H[1][0]*0 + H[1][1]*size[0] + H[1][2])/(H[2][0]*0 + H[2][1]*size[0] + H[2][2])))
    x.append(((H[0][0]*size[1] + H[0][1]*0 + H[0][2])/(H[2][0]*size[1] + H[2][1]*0 + H[2][2])))
    y.append(((H[1][0]*size[1] + H[1][1]*0 + H[1][2])/(H[2][0]*size[1] + H[2][1]*0 + H[2][2])))
    x.append(((H[0][0]*size[1] + H[0][1]*size[0] + H[0][2])/(H[2][0]*size[1] + H[2][1]*size[0] + H[2][2])))
    y.append(((H[1][0]*size[1] + H[1][1]*size[0] + H[1][2])/(H[2][0]*size[1] + H[2][1]*size[0] + H[2][2])))

    min_x = min(x)
    min_y = min(y)
    max_x = max(x)
    max_y = max(y)
    div = ((min_x, max_x),(min_y, max_y))
    return div

def resize_mat(image, div):
    height, width = image.shape[0:2]
    d = [0, 0, width, height]
    if div[0][0] < 0:
        d[0] = div[0][0]
    if div[0][1] > width:
        d[2] = div[0][1]
    if div[1][0] < 0:
        d[1] = div[1][0]
    if div[1][1] > height:
        d[3] = div[1][1]
    T = np.array([[1.0, 0.0, -d[0]], [0.0, 1.0, -d[1]], [0.0, 0.0, 1.0]])
    image = cv2.warpPerspective(image, T, (int(-d[0] + d[2]), int(-d[1] + d[3])))
    return (image,d)


def get_center(mask):
    min_x = 10000
    max_x = -1
    min_y = 10000
    max_y = -1
    for y in range(mask.shape[0]):
        for x in range(mask.shape[1]):
            if(mask[y][x]):
                if(x<min_x):
                    min_x = x
                if(y<min_y):
                    min_y = y
                if(x>max_x):
                    max_x = x
                if(y>max_y):
                    max_y = y
    return (max_y+min_y)/2, (max_x+min_x)/2


def arrange_rgb(mat, TargetMask):
    mat[TargetMask==0] = [0,0,0]
    gray = cv2.cvtColor(mat,cv2.COLOR_RGB2GRAY)
    mat[(TargetMask != 0) * (gray == 0)] = 1
    return mat

def write_blending(target, source, SrcMask):
    mask = cv2.cvtColor(SrcMask,cv2.COLOR_GRAY2RGB)
    target[(mask != [0,0,0])] = source[(mask != [0,0,0])]
    return target

In [33]:
class Stitcher:
    def __init__(self):
        # determine if we are using OpenCV v3.X
        self.isv3 = imutils.is_cv3()

    def stitch(self, images, ratio=0.75, reprojThresh=4.0,
               showMatches=False):
        # unpack the images, then detect keypoints and extract
        # local invariant descriptors from them
        (imageB, imageA) = images
        (kpsA, featuresA) = self.detectAndDescribe(imageA)
        (kpsB, featuresB) = self.detectAndDescribe(imageB)

        # match features between the two images
        M = self.matchKeypoints(kpsA, kpsB,
                                featuresA, featuresB, ratio, reprojThresh)

        # if the match is None, then there aren't enough matched
        # keypoints to create a panorama
        if M is None:
            return None

        # otherwise, apply a perspective warp to stitch the images
        # together
        (matches, H, status) = M
        
        #result = cv2.warpPerspective(imageA, H,
         #                            (imageA.shape[1] + imageB.shape[1], imageA.shape[0] + imageB.shape[0]))
     #########################
        div = calc_dst4points(H, imageA.shape) #original1.image-->imageB, original2..-->imageA, imageB.image-->imageB
        imageB,d = resize_mat(imageB,div)        
        T_xy = [[1.0, 0.0, -d[0]],[0.0, 1.0, -d[1]],[0.0, 0.0, 1.0]]
        panorama = cv2.warpPerspective(imageA,np.dot(T_xy,H),(imageB.shape[1],imageB.shape[0]))
        CommonMask, SrcMask, TargetMask = make_mask(panorama, imageB)
        label = cv2.connectedComponentsWithStats(CommonMask)
        center = np.delete(label[3], 0, 0)
        test = get_center(CommonMask)
        blending = cv2.seamlessClone(imageB, panorama, cv2.cvtColor(CommonMask,cv2.COLOR_GRAY2BGR), (int(test[1]),int(test[0])), cv2.NORMAL_CLONE)
        blending = arrange_rgb(blending, TargetMask)
        blending = write_blending(blending, imageB, SrcMask)
        result=blending
        #result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
#         for i in range(imageB.shape[0]):  #这里改写了，原代码result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
#             for j in range(imageB.shape[1]):
#                 if not (imageB[i][j] == [0, 0, 0]).all():
#                     result[i][j] = imageB[i][j]  #result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

        # check to see if the keypoint matches should be visualized
        if showMatches:
            vis = self.drawMatches(imageA, imageB, kpsA, kpsB, matches,  #
                                   status)

            # return a tuple of the stitched image and the
            # visualization
            return (result, vis, H)  #########这里改动

        # return the stitched image
        return (result, H)


    def detectAndDescribe(self, image):
        # convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # check to see if we are using OpenCV 3.X
        if self.isv3:
            # detect and extract features from the image
            descriptor = cv2.xfeatures2d.SIFT_create()
            (kps, features) = descriptor.detectAndCompute(image, None)

        # otherwise, we are using OpenCV 2.4.X
        else:
            # detect keypoints in the image
            detector = cv2.FeatureDetector_create("SIFT")
            kps = detector.detect(gray)

            # extract features from the image
            extractor = cv2.DescriptorExtractor_create("SIFT")
            (kps, features) = extractor.compute(gray, kps)

        # convert the keypoints from KeyPoint objects to NumPy
        # arrays
        kps = np.float32([kp.pt for kp in kps])

        # return a tuple of keypoints and features
        return (kps, features)
    
    



    def matchKeypoints(self, kpsA, kpsB, featuresA, featuresB,
                       ratio, reprojThresh):
        # compute the raw matches and initialize the list of actual
        # matches
         # FLANN parameters这一段是flann match,换一个match
#         FLANN_INDEX_KDTREE = 0
#         index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
#         search_params = dict(checks=50)   # or pass empty dictionary
#         matcher = cv2.FlannBasedMatcher(index_params,search_params)

        ##################
        matcher = cv2.DescriptorMatcher_create("BruteForce")
        
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
        matches = []
        #         matches = sorted(matches, key = lambda x:x.distance)
        # loop over the raw matches
        for m in rawMatches:
            # ensure the distance is within a certain ratio of each
            # other (i.e. Lowe's ratio test)
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))
        # computing a homography requires at least 4 matches
        if len(matches) > 4:
            # construct the two sets of points
            ptsA = np.float32([kpsA[i] for (_, i) in matches[:10]])  #这里可以对matches特征数量加以限制，如matches[:100]
            ptsB = np.float32([kpsB[i] for (i, _) in matches[:10]])
        
        ######下面这段网格筛选
#             ptsA=ptsA.tolist()
#             ptsB=ptsB.tolist()
#             ptsA,ptsB = grid_pick(ptsA,ptsB)
#             ptsA,ptsB = np.array(ptsA), np.array(ptsB)
            #compute the homography between the two sets of points
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,
                                             reprojThresh)

            # return the matches along with the homograpy matrix
            # and status of each matched point
            return (matches, H, status)

        # otherwise, no homograpy could be computed
        return None

    def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
        # initialize the output visualization image
        (hA, wA) = imageA.shape[:2]
        (hB, wB) = imageB.shape[:2]
        vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        vis[0:hA, 0:wA] = imageA
        vis[0:hB, wA:] = imageB

        # loop over the matches
        for ((trainIdx, queryIdx), s) in zip(matches, status):
            # only process the match if the keypoint was successfully
            # matched
            if s == 1:
                # draw the match
                ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
                ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

        # return the visualization
        return vis


In [24]:
def gene1crop(pic1path,pic2path,height=1000):
    imageB = cv2.imread(pic1path)
    imageA = cv2.imread(pic2path)
    #imageB = imutils.resize(imageB, height)
    imageA = imutils.resize(imageA, height)
    # stitch the images together to create a panorama
    stitcher = Stitcher()
    (result, vis,H) = stitcher.stitch([imageB, imageA], showMatches=True)

    # show the images
    # cv2.imshow("Image A", imageA)
    # cv2.imshow("Image B", imageB)
    #
    #cv2.imshow("Result", result)
    #cv2.imwrite("0102bgtst.jpg",result)
    #按图像大小裁剪一个框，去掉框外黑区域
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    hmax=0
    wmax=0
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            if gray[i][j]!=0:
                if wmax<j:wmax=j
                if hmax<i:hmax=i
    cropImg = result[0:hmax, 0:wmax]
    
    return (cropImg,vis,H)

def gene1img(pic1path,pic2path,height=1000): #pic1path是PosixPath对象，可以用.parts操作
    cropImg,vis,H=gene1crop(str(pic1path),str(pic2path),height=height)
    cv2.imwrite("test01.jpg",cropImg)
    cv2.imwrite("green01.jpg",vis)
    ###下面添加dump H矩阵代码，保存名字是dump_heat+第二张图的文件名
    inname=pic2path.parts[-1].split(".")[0]
    dumpname='dump'+inname+'.txt' #保存H矩阵
    f = open(dumpname, 'wb')
    pickle.dump(H, f)
    f.close()
    #######
    return "test01.jpg"

In [7]:
#gene1img("./tmp/folderImg/010203_1000.jpg","./tmp/folderImg/04.jpg",height=1000)

'test01.jpg'

imageT = imutils.resize('./bcropsamp.jpg', 800)
cv2.imwrite("imageT.jpg",imageT)

gene1img('bcropsamp.jpg','04.jpg')

### 以上是函数

In [28]:
PATH = Path('./')
folderImg=PATH.glob('./tmp/folderImg/notused/*.jpg') #./tmp/folderImg3/*.jpg
imglist=[i for i in folderImg]
imglist=sorted(imglist)
imglist

[PosixPath('tmp/folderImg/notused/01.jpg'),
 PosixPath('tmp/folderImg/notused/02.jpg')]


images=os.listdir(folderImg)
images=sorted(images)
images

In [7]:
imglist[0].parts[-1].split(".")[0]

'01'

imageB = cv2.imread(str(imglist[0]))

In [34]:

reduce(gene1img, imglist)

'test01.jpg'

In [ ]:
imageB = cv2.imread('tmp/folderImg1/01.jpg')
# imageB

In [ ]:

# print(imageB.smhape)

imageB = cv2.imread('01.jpg')
imageA = cv2.imread('02.jpg')
imageB = imutils.resize(imageB, height=800)
imageA = imutils.resize(imageA, height=800)
# stitch the images together to create a panorama
stitcher = Stitcher()
(result, vis,H) = stitcher.stitch([imageB, imageA], showMatches=True)
 
# show the images
# cv2.imshow("Image A", imageA)
# cv2.imshow("Image B", imageB)
#
#cv2.imshow("Result", result)
cv2.imwrite("samp.jpg",result)
cv2.waitKey(0)

#### 把拼出的图像裁剪


img = cv2.imread('010203bgtst.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
hmax=0
wmax=0
for i in range(gray.shape[0]):
    for j in range(gray.shape[1]):
        if gray[i][j]!=0:
            if wmax<j:wmax=j
            if hmax<i:hmax=i
hmax,wmax


cropImg = img[0:hmax, 0:wmax]
cv2.imwrite("b.jpg",cropImg)

imageB = cv2.imread('b.jpg')
imageA = cv2.imread('04.jpg')
imageB = imutils.resize(imageB, height=800)
imageA = imutils.resize(imageA, height=800)
# stitch the images together to create a panorama
stitcher = Stitcher()
(result, vis,H) = stitcher.stitch([imageB, imageA], showMatches=True)
 
# show the images
# cv2.imshow("Image A", imageA)
# cv2.imshow("Image B", imageB)
#
cv2.imshow("Green lines", vis)
cv2.imwrite("01020304bgtstlines.jpg",vis)
#cv2.imshow("Result", result)
#cv2.imwrite("01020304bgtst.jpg",result)
cv2.waitKey(0)

### 这里整合上面几段缝合代码，写循环，实现多图连拼

---

imageB = cv2.imread('010203bg001.jpg')
imageA = cv2.imread('04.jpg')
imageB = imutils.resize(imageB, height=800)
imageA = imutils.resize(imageA, height=800)
# stitch the images together to create a panorama
stitcher = Stitcher()
(result, vis,H) = stitcher.stitch([imageB, imageA], showMatches=True)
 
# show the images
# cv2.imshow("Image A", imageA)
# cv2.imshow("Image B", imageB)
#
#cv2.imshow("Result", result)
cv2.imwrite("01020304bg001.jpg",result)
cv2.waitKey(0)

imageB1 = cv2.imread('bg1.jpg')
imageA1 = cv2.imread('bg2.jpg')
result1 = cv2.warpPerspective(imageA1, H,
            (imageA1.shape[1] + imageB1.shape[1], imageA1.shape[0]))
result1[0:imageB1.shape[0], 0:imageB1.shape[1]] = imageB1

cv2.imwrite("0102bg1.jpg",result1)
